<a href="https://colab.research.google.com/github/sanjanams742/sanjanams742/blob/main/cyberbullying_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PROJECT TITLE: Cyberbullying Classification**

**Visualization and Prediction using NLP**

As social media usage becomes increasingly prevalent in every age group, a vast majority of citizens rely on this essential medium for day-to-day communication. Social media’s ubiquity means that cyberbullying can effectively impact anyone at any time or anywhere, and the relative anonymity of the internet makes such personal attacks more difficult to stop than traditional bullying.

On April 15th, 2020, UNICEF issued a warning in response to the increased risk of cyberbullying during the COVID-19 pandemic due to widespread school closures, increased screen time, and decreased face-to-face social interaction. The statistics of cyberbullying are outright alarming: 36.5% of middle and high school students have felt cyberbullied and 87% have observed cyberbullying, with effects ranging from decreased academic performance to depression to suicidal thoughts.

In light of all of this, this dataset contains more than 47000 tweets labelled according to the class of cyberbullying:

Age
Ethnicity
Gender
Religion
Other type of cyberbullying
Not cyberbullying

Trigger Warning: These tweets either describe a bullying event or are the offense themselves, therefore explore it to the point where you feel comfortable.

In this project, I'm going to analyze the tweets, plot some visualizations, process them with different NLP techniques like TFIDF and Word2Vec and finally evaluate predictions of various ML Classifiers and RNN.

Let's get started 😇

**Importing Libraries**

In [ ]:
# Utility
import numpy as np
import pandas as pd
import pickle

# Plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

unzip:  cannot find or open /usr/share/nltk_data/corpora/wordnet.zip, /usr/share/nltk_data/corpora/wordnet.zip.zip or /usr/share/nltk_data/corpora/wordnet.zip.ZIP.


**Data Import**

In [ ]:
df = pd.read_csv('/content/cyberbullying_tweets.csv')
display(df.sample(5))
df.shape

,tweet_text,cyberbullying_type
10871,@DrJoanBeckwith You don't regard Ford as a pro...,gender
20902,"By the way, how the hell u have compared @RSSo...",religion
3926,RT @NamAndKang: [VIDEO] WINNER #ìë - MelOn...,not_cyberbullying
45231,.@tylerrjoseph follow me on ur stan acc asf u ...,ethnicity
19262,I just had flashbacks to when a Jamaican girl ...,religion


(47692, 2)

We can see there are 47692 tweets in total.

In [ ]:
df['cyberbullying_type'].unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

They belong to 6 different classes, which are:

Age, Gender, Ethnicity, Religion, Other and Not Cyberbullying

In [ ]:
for tweet in df.sample(5)['tweet_text']:
    print(tweet +'\n')

@BilalIGhumman @98Halima @johnnygjokaj @cdnKhadija @rfrankh53 The stupidist arguments are the ones rationalizing the barbarity of Islam.

@Bo_OYoww: “@tayyoung_: FUCK OBAMA, dumb ass nigger”-- &amp; I guess you mad cuz you live on a farm lmao

Queensland schools are cautiously watching the second wave of coronavirus outbreaks in southern states and bracing for a future of remote learning.

@ctatplay @erinscafe @Mobute shhhh.

The closest thing we have is Progressive policy. Read up on the founding of the Progressive movement. All White supremacists. They created policy like 'The Negro Project' (later called Planned Parenthood) to keep minority births down. The adopted the minimum wage to ensure....



This is what the tweets look like. Pretty toxic.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          47692 non-null  object
 1   cyberbullying_type  47692 non-null  object
dtypes: object(2)
memory usage: 745.3+ KB


In [ ]:
df.describe()

,tweet_text,cyberbullying_type
count,47692,47692
unique,46017,6
top,RT @sailorhg: the intro for my hardware hackin...,religion
freq,2,7998


**Data Pre-processing**

 Are there any null or duplicate values?

In [ ]:
df.isnull().sum()

,0
tweet_text,0
cyberbullying_type,0


In [ ]:
df['tweet_text'].duplicated().sum()

1675

Let's drop these duplicated.

In [ ]:
df.drop_duplicates('tweet_text', inplace=True, ignore_index=True)
df['tweet_text'].duplicated().sum()

0

Are the classes balanced?

In [ ]:
df['cyberbullying_type'].value_counts()

,count
cyberbullying_type,
religion,7995
age,7992
ethnicity,7952
not_cyberbullying,7937
gender,7898
other_cyberbullying,6243


Looks pretty balanced.

Since the other_cyberbullying class is too generic, contains many categories in itself and brings down the accuracies of the upcoming prediction models, I have decided to remove it and work with the remaining classes.

In [ ]:
df = df[df["cyberbullying_type"] != "other_cyberbullying"]
df['cyberbullying_type'].unique()

array(['not_cyberbullying', 'gender', 'religion', 'age', 'ethnicity'],
      dtype=object)

Label Encoding the Classes




In [ ]:
cyberbullying_type = ['not_cyberbullying', 'gender', 'religion', 'age', 'ethnicity']
encoding_dict = {'not_cyberbullying':0, 'gender':1, 'religion':2, 'age':3, 'ethnicity':4}
df['cyberbullying_type'] = df['cyberbullying_type'].map(encoding_dict)
df.sample(5)

,tweet_text,cyberbullying_type
11751,"And yet somehow, they thrive without your igno...",1
12950,I h8 you if : 1) you make rape jokes 2) you're...,1
33446,(2/2) i probably wont be able to go to high sc...,3
34944,i was buillied by a girl for a whole ass year ...,3
18099,it was about muslims you abs idiot CAN YOU REA...,2


NLP Processing

Time to clean the tweets and process them to a new column.

In [ ]:
# Downloading dependencies
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
stopwords_list = stopwords.words('english')
additional_stopwords = ['rt', 'mkr', 'didn', 'bc', 'n', 'm', 'im', 'll', 'y', 've', 'u', 'ur', 'don', 't', 's']   # Specific to twitter lingo
stopwords_list.extend(additional_stopwords)
lemmatizer = WordNetLemmatizer()    # Using Lemmatizer instead of Stemmer

Below are the custom defined functions to deep clean the unprocessed tweets.
Their identifier names describe their working.

In [ ]:
def remove_usernames(tweet):
    words = tweet.strip().split()
    words = [word for word in words if word[0]!='@']
    return ' '.join(words)

def remove_hashtags(tweet):
    words = tweet.strip().split()
    words = [word for word in words if word[0]!='#']
    return ' '.join(words)

def remove_links(tweet):
    words = tweet.strip().split()
    words = [word for word in words if word[:4]!='http']
    return ' '.join(words)

def remove_punctuation(tweet):
    for mark in punctuation:
        tweet = tweet.replace(mark, '')
    return tweet

def convert_lowercase(tweet):
    return tweet.lower()

def tokenization(tweet):
    return word_tokenize(tweet)

def remove_stopwords(tok_tweet):
    res = []
    for word in tok_tweet:
        if word not in stopwords_list:
            res.append(word)
    return res

def lemmatize_words(tok_tweet):
    res = []
    for word in tok_tweet:
        res.append(lemmatizer.lemmatize(word))
    return res

In [ ]:
# Defining a custom tweet cleaning function
def pre_processing_custom(tweet):
    tweet = remove_usernames(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_links(tweet)
    tweet = remove_punctuation(tweet)
    tweet = convert_lowercase(tweet)
    tok_tweet = tokenization(tweet)
    tok_tweet = remove_stopwords(tok_tweet)
    tok_tweet = lemmatize_words(tok_tweet)
    clean_tweet = ' '.join(tok_tweet)

    return clean_tweet

Time to apply this processing function to our dataset.

In [ ]:
df['clean_tweets'] = df['tweet_text'].map(pre_processing_custom)
df.sample(5)

,tweet_text,cyberbullying_type,clean_tweets
41332,You twitter niggers are dumb as fuck.,4,twitter nigger dumb fuck
2430,#confessionnight i'm friends with no one I use...,0,friend one use back middle school
32589,I remember one time she talked about getting p...,3,remember one time talked getting permission th...
42177,Don't sleep with me? Then don't speak with me....,4,dont sleep dont speak never talk bad bout nigg...
38727,HOW THE FUCK IS THIS BITCH GUNNA TELL ME TO GO...,4,fuck bitch gunna tell go hallway dumb as nigge...


The clean_tweets column holds the processed tweets.

It's nice and does the job, but we can do much better cleaning by using pre-defined functions and RegEx.
Therefore, I'm using the tweet-preprocessor library, as shown below.

In [ ]:
!pip install tweet-preprocessor
import preprocessor as p

In [ ]:
def clean_tweets(tweet):
    tweet = p.clean(tweet)
    return tweet

It takes care of all the URLs, Hashtags, Mentions, Reserved Words, Emojis and Smileys.

In [ ]:
# Defining the better cleaning function
def pre_processing(tweet):
    tweet = clean_tweets(tweet)
    tweet = remove_punctuation(tweet)
    tweet = convert_lowercase(tweet)
    tok_tweet = tokenization(tweet)
    tok_tweet = remove_stopwords(tok_tweet)
    tok_tweet = lemmatize_words(tok_tweet)
    clean_tweet = ' '.join(tok_tweet)

    return clean_tweet

In [ ]:
# Clean tweets
df['clean_tweets'] = df['tweet_text'].map(pre_processing)
df.sample(5)

,tweet_text,cyberbullying_type,clean_tweets
18692,More complaining from the Five Time Draft Dodg...,2,complaining five time draft dodger far right r...
16069,It’s Friday again. Why is it so hard for terro...,2,friday hard terrorist resist jihad jumma day m...
8333,and he used gay as an insult and made rape jok...,1,used gay insult made rape joke yikes
8473,Empowered female reporter for Al Jazeera (Kimb...,1,empowered female reporter al jazeera kimberly ...
22755,"2.@narendramodi,@AmitShah, @HMOIndiaWITH POLIT...",2,political support conversion rohingyas confirm...


Checking again if any values got duplicated.

In [ ]:
df['clean_tweets'].duplicated().sum()

1217

Removing them all.

In [ ]:
df.drop_duplicates('clean_tweets', inplace=True, ignore_index=True)
df['clean_tweets'].duplicated().sum()

0

38557 tweets are remaining.

Now that we have processed the tweets, it's time to move forward and calculate the total and average word lengths of each tweet.

In [ ]:
df.shape

(38557, 3)

In [ ]:
# Total word count of a tweet
df['word_count'] = df['tweet_text'].map(lambda x: len(x.split()))
df.sample(5)

,tweet_text,cyberbullying_type,clean_tweets,word_count
3610,This having class on Monday and Tuesday thing ...,0,class monday tuesday thing quite homosexual,11
12260,Yeah shit 12 year olds are not qualified at al...,1,yeah shit year old qualified sexist would neve...,23
5562,"acabei de terminar a redação de portugues, o a...",0,acabei de terminar redao de portugues assunto ...,22
14523,"Fuck it, i'm young. No joke, i'm 69 and gay. R...",1,fuck young joke gay rape,12
26993,That babcock stuff makes me so fucking angry w...,3,babcock stuff make fucking angry manipulative ...,33


In [ ]:
#import pandas as pd
#from google.colab import files

# Save the DataFrame to a CSV file in the Colab environment
#df.to_csv('cleaned_dataset.csv', index=False)

# Download the file to your local system
#files.download('cleaned_dataset.csv')

In [ ]:
# Average word length of a tweet
def avg_word_length(tweet):
    words = tweet.split()
    word_count = len(words)
    word_length = 0
    for word in words:
        word_length += len(word)
    avg_length = round(word_length / word_count,2)
    return avg_length

df['avg_word_length'] = df['tweet_text'].map(avg_word_length)
df.sample(5)

,tweet_text,cyberbullying_type,clean_tweets,word_count,avg_word_length
17775,You just juggle topics to suit your narrative ...,2,juggle topic suit narrative really pathetic ta...,48,4.52
19824,Deplorable is too nice a word for you GABI and...,2,deplorable nice word gabi bet bible thumping c...,31,3.87
29434,middle school bullies were the worst bullies,3,middle school bully worst bully,7,5.43
5346,"@iFalasteen No, this is what Muslim brutality ...",0,muslim brutality look like,10,7.10
15433,Listen. There's a Christian woman boldly procl...,2,listen there christian woman boldly proclaimin...,40,4.92


We can also remove the outliers and the tweets useless to us from this dataset, like this

In [ ]:
df = df[df['word_count']>3]
df = df[df['word_count']<100]
df.reset_index(drop=True, inplace=True)
df.shape

(38201, 5)

In [ ]:
import nltk.sentiment
nltk.download('vader_lexicon')

def get_score(input_text):
    analyzer = nltk.sentiment.SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(input_text)
    sentiment_score = scores['compound']
    return sentiment_score

def get_reaction(score):
    if(score > 0.5):
        return 'Highly Positive'
    if(score > 0):
        return 'Positive'
    if(score == 0):
        return 'Neutral'
    if(score < -0.5):
        return 'Highly Negative'
    if(score < 0):
        return 'Negative'

def main():
    input_text = input("Please enter a sentence : ")
    score = get_score(input_text)
    reaction = get_reaction(score)
    print('Sentiment score : ', score)
    print(reaction)
if __name__ == "__main__":
    main()


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


**Data Visualization**

In [ ]:
sns.set_theme()
plt.figure(figsize=(10, 5))
ax = sns.countplot(data=df, y='cyberbullying_type', palette="viridis")
ax.bar_label(ax.containers[0])
plt.yticks(ticks=[0,1,2,3,4], labels=cyberbullying_type)
plt.xlabel('Tweets Count', size=15)
plt.ylabel('Type', size=15)
plt.grid(color='w')

All the classes look balanced.

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, y='cyberbullying_type', x='word_count', orient='h', showfliers=False,
            palette='viridis', notch=True, medianprops={"color": "coral"})
plt.yticks(ticks=[0,1,2,3,4], labels=cyberbullying_type)
plt.xlabel('Word Length', size=15)
plt.ylabel('Type', size=15)
plt.grid(color='w')

We can observe that the tweets targeted towards a gender tend to have a bit less words in them than say, the ones targeted towards religion.
But overall, a bullying tweet has more words in it and is more varied than a normal one.

In [ ]:
plt.figure(figsize=(8, 5))
sns.violinplot(data=df, y='cyberbullying_type', x='avg_word_length', orient='h', palette='mako')
plt.xlim(0, 10)
plt.yticks(ticks=[0,1,2,3,4], labels=cyberbullying_type)
plt.xlabel('Average Word Length', size=14)
plt.ylabel('Type', size=15)
plt.grid(color='w')

Pretty much the same for all, between 4 to 6 letters.

In [ ]:
plt.figure(figsize=(8, 5))
sns.jointplot(data=df, y='avg_word_length', x='word_count', hue='cyberbullying_type', palette='rainbow')
plt.xlabel('Total Words', size=15)
plt.ylabel('Average Word Length', size=14)

**Word Clouds**

In [ ]:
# Gender
plt.figure(figsize=(10, 10))
subset1 = df[df['cyberbullying_type']==1]
text_gender = subset1['clean_tweets'].values
wc1 = WordCloud(background_color='white', colormap='Dark2', width=2000, height=1000).generate(' '.join(text_gender))
plt.axis('off')
plt.title("Gender", fontsize=20)
plt.imshow(wc1, interpolation='bilinear')

In [ ]:
# Religion
plt.figure(figsize=(10, 10))
subset2 = df[df['cyberbullying_type']==2]
text_religion = subset2['clean_tweets'].values
wc2 = WordCloud(background_color='white', colormap='Dark2', width=2000, height=1000).generate(' '.join(text_religion))

plt.axis('off')
plt.title("Religion", fontsize=20)
plt.imshow(wc2, interpolation='bilinear')

In [ ]:
# Age
plt.figure(figsize=(10, 10))
subset3 = df[df['cyberbullying_type']==3]
text_age = subset3['clean_tweets'].values
wc3 = WordCloud(background_color='white', colormap='Dark2', width=2000, height=1000).generate(' '.join(text_age))

plt.axis('off')
plt.title("Age", fontsize=20)
plt.imshow(wc3, interpolation='bilinear')

In [ ]:
# Ethnicity
plt.figure(figsize=(10, 10))
subset4 = df[df['cyberbullying_type']==4]
text_ethnicity = subset4['clean_tweets'].values
wc4 = WordCloud(background_color='white', colormap='Dark2', width=2000, height=1000).generate(' '.join(text_ethnicity))

plt.axis('off')
plt.title("Ethnicity", fontsize=20)
plt.imshow(wc4, interpolation='bilinear')

In [ ]:
# Not Bullying
plt.figure(figsize=(10, 10))
subset0 = df[df['cyberbullying_type']==0]
text_not = subset0['clean_tweets'].values
wc0 = WordCloud(background_color='white', colormap='Dark2', width=2000, height=1000).generate(' '.join(text_not))

plt.axis('off')
plt.title("Not Bullying", fontsize=20)
plt.imshow(wc0, interpolation='bilinear')

In [ ]:
# All Tweets
plt.figure(figsize=(10, 10))
text_all = df['clean_tweets'].values
wc = WordCloud(background_color='white', colormap='Dark2', width=2000, height=1000).generate(' '.join(text_all))

plt.axis('off')
plt.title("All Tweets", fontsize=20)
plt.imshow(wc, interpolation='bilinear')

Now that we're done with the wordclouds, let's check which words are the most popular.

In [ ]:
from collections import Counter   # To count the frequency of each word

temp=[]
for tweet in df['clean_tweets']:
  for word in tweet.split():
    temp.append(word)
vocab = Counter(temp)
print("Vocabulary Size:", len(vocab))
print(list(vocab.items())[:5])

vocab contains the frequency of all the unique words in the whole dataset. Pretty interesting.
Let's try to plot it and see what we get.

In [ ]:
common_word=[]
freq=[]
for i, j in vocab.most_common()[:10]:
  common_word.append(i)
  freq.append(j)

In [ ]:
# Most common words
plt.figure(figsize=(10, 5))
ax = sns.barplot(x=common_word, y=freq, palette='viridis')
plt.title('Top 10 most common words', size=20)
ax.bar_label(ax.containers[0])
plt.ylabel("Word Length", size=15)
plt.grid()

In [ ]:
from transformers import pipeline

# Load pre-trained emotion detection model
emotion_model = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

# Function to detect emotions
def detect_emotions(texts):
    # Pass the input text to the model
    predictions = emotion_model(texts)

    # Collect emotions and scores
    results = []
    for i, prediction in enumerate(predictions):
        emotions = {item['label']: item['score'] for item in prediction}
        sorted_emotions = sorted(emotions.items(), key=lambda x: x[1], reverse=True)
        results.append({
            "text": texts[i],
            "top_emotion": sorted_emotions[0][0],  # Most likely emotion
            "emotion_scores": emotions             # All emotions with their respective scores
        })
    return results

# Test the function with some sample texts
texts = [
    "feeling perturbed woman arent humble kindly go set fire"
    "Rebecca Black Drops Out of School Due to Bullying",
    "meh thanks head concerned another angry dude twitter",
    "This is so frustrating and annoying!",
    "Wow, I'm really surprised by this amazing news!",
    "I'm feeling anxious about tomorrow."
]

# Get emotion predictions
emotion_results = detect_emotions(texts)

# Display the results
for result in emotion_results:
    print(f"Text: {result['text']}")
    print(f"Top Emotion: {result['top_emotion']}")
    print(f"Emotion Scores: {result['emotion_scores']}")
    print("-" * 50)


**ML Classifiers**

Now that we have analyzed and visualized the dataset, had some reality check after seeing the wordclouds, its time to get our hands dirty and apply various ML classifiers to it, and see which one performs the best.

First, we start by splitting the data into train and test.

In [ ]:
X = df['clean_tweets']
y = df['cyberbullying_type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=76)

TF-IDF Vectorization

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,2))
tfidf.fit(X_train)    # Fit on train
print("No. of feature words:", len(tfidf.get_feature_names_out()))

In [ ]:
X_train_tf = tfidf.transform(X_train)
X_test_tf = tfidf.transform(X_test)
print(f"{X_train_tf.shape = }\n{X_test_tf.shape = }")

We're ready now, time to apply the models.

**Logistic Regression**

In [ ]:
# Model training
lr = LogisticRegression(solver='saga', penalty='l2', random_state=394)
lr.fit(X_train_tf, y_train)

In [ ]:
# Model prediction
y_pred_train = lr.predict(X_train_tf)
y_pred_test = lr.predict(X_test_tf)

Our Logistic Regression Model is ready, let's evaluate it.

In [ ]:
# Model evaluation
print('\tClassification Report for Logistic Regression on Train\n\n', classification_report(y_train, y_pred_train, target_names=cyberbullying_type))
print('\n\tClassification Report for Logistic Regression on Test\n\n', classification_report(y_test, y_pred_test, target_names=cyberbullying_type))

The performance scores of the algorithm are very good, with an overall accurcy of 92%.
We can observe how the predictions for the classes have very high F1 scores, while for the class "non-cyberbullying" the score is much lower.

In [ ]:
# Confusion Matrix for Logistic Regression
sns.heatmap(confusion_matrix(y_test, y_pred_test), annot=True, cmap="Greens", fmt='', cbar=False)
plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
plt.ylabel("Actual values", fontdict = {'size':14}, labelpad = 10)
plt.title('Confusion Matrix LR', size=15)

Let's try to predict on other models also, then we can compare.

**Naive-Bayes**

In [ ]:
# Model training
mnb = MultinomialNB(alpha=0.1)
mnb.fit(X_train_tf, y_train)

In [ ]:
# Model prediction
y_pred_train = mnb.predict(X_train_tf)
y_pred_test = mnb.predict(X_test_tf)

In [ ]:
# Model evaluation
print('\tClassification Report for Naive-Bayes\' on Train\n\n', classification_report(y_train, y_pred_train, target_names=cyberbullying_type))
print('\n\tClassification Report for Naive-Bayes\' on Test\n\n', classification_report(y_test, y_pred_test, target_names=cyberbullying_type))

This clearly looks like a case of overfitting, or high variance, and Naive-Bayes' is famous for it.
Again we can see the 'not_cyberbullying' class performing worst, as it's too generic.

Let's see if we could resolve this issue in the next model, i.e., Random Forest.

In [ ]:
# Confusion Matrix for Naive-Bayes'
sns.heatmap(confusion_matrix(y_test, y_pred_test), annot=True, cmap="Greens", fmt='', cbar=False)
plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
plt.ylabel("Actual values", fontdict = {'size':14}, labelpad = 10)
plt.title('Confusion Matrix NB', size=15)

**Random Forest**

In [ ]:
# Model training
rf = RandomForestClassifier(n_estimators=100, criterion="entropy", max_depth=30, random_state=27)
rf.fit(X_train_tf, y_train)

Applied hyperparameter tuning using GridSearchCV below to find the best parameters for our model.

I've commented this block out as it takes too much time to run.
From here we can get the best parameters to use with Random Forest. I've applied them already.

In [ ]:
# Model prediction
y_pred_train = rf.predict(X_train_tf)
y_pred_test = rf.predict(X_test_tf)

It did reduce the overfitting issue and gave an impressive accuracy of 90% also.

In [ ]:
# Model evaluation
print('\tClassification Report for Random Forest on Train\n\n', classification_report(y_train, y_pred_train, target_names=cyberbullying_type))
print('\n\tClassification Report for Random Forest on Test\n\n', classification_report(y_test, y_pred_test, target_names=cyberbullying_type))

In [ ]:
# Confusion Matrix for Random Forest
sns.heatmap(confusion_matrix(y_test, y_pred_test), annot=True, cmap="Greens", fmt='', cbar=False)
plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
plt.ylabel("Actual values", fontdict = {'size':14}, labelpad = 10)
plt.title('Confusion Matrix RF', size=15)

So far, Logistic Regression has performed the best with an amazing 92% accuracy.
Random Forest coming in at a close second with 90% accuracy overall.

Ensemble

We're going to use Voting technique here to aggregate the predictions, in order to improve the performance of our model.

In [ ]:
# Model Training
from sklearn.ensemble import VotingClassifier

vot = VotingClassifier(estimators = [('lr', lr), ('mnb', mnb), ('rf', rf)], voting='soft')
vot.fit(X_train_tf, y_train)

In [ ]:
# Model prediction
y_pred_train = vot.predict(X_train_tf)
y_pred_test = vot.predict(X_test_tf)

In [ ]:
# Model evaluation
print('\tClassification Report for Voting on Train\n\n', classification_report(y_train, y_pred_train, target_names=cyberbullying_type))
print('\n\tClassification Report for Voting on Test\n\n', classification_report(y_test, y_pred_test, target_names=cyberbullying_type))

As good as it can get. 91% accuracy on test data. We're using this model for all our future predictions.

In [ ]:
# Confusion Matrix after Voting
sns.heatmap(confusion_matrix(y_test, y_pred_test), annot=True, cmap="Greens", fmt='', cbar=False)
plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
plt.ylabel("Actual values", fontdict = {'size':14}, labelpad = 10)
plt.title('Confusion Matrix Voting', size=15)

Except 'not_bullying', all of our classes have been predicted well, with over 90% accuracy.

Now our final task is to get our model up and running, and use it to predict on some random texts.

**Model Saving, Loading and Prediction**

We can save our model, so it can be used whenever we require.
Here I have used pickle library for saving and loading.

In [ ]:
# Saving Logistic Regression Model
model_file = open('LogisticRegression.pickle','wb')
pickle.dump(lr, model_file)
model_file.close()

# Saving Naive-Bayes' Model
model_file = open('NaiveBayes.pickle','wb')
pickle.dump(mnb, model_file)
model_file.close()

# Saving Random Forest Model
model_file = open('RandomForest.pickle','wb')
pickle.dump(rf, model_file)
model_file.close()

# Saving our final voting aggregated model
model_file = open('Voting.pickle','wb')
pickle.dump(vot, model_file)
model_file.close()

Prediction using saved model

In [ ]:
# Defing our custom prediction function
def predict(model, texts):
    clean_texts = [pre_processing_custom(text) for text in texts]
    text_data = tfidf.transform(clean_texts)
    prediction = model.predict(text_data)

    data = []
    for text, prediction in zip(texts, prediction):
        data.append((text, prediction))

    df = pd.DataFrame(data, columns = ['text','type'])
    df = df.replace([0,1,2,3,4], cyberbullying_type)
    return df

Prediction

In [ ]:
# Using our best model
model_file = open('Voting.pickle', 'rb')
model = pickle.load(model_file)
model_file.close()

# Providing some random toxic texts
texts = ["I hate you bitch",
        "School shooting. Haha dead.",
        "Treat them like black slaves",
        "I don't feel so good Mr. Potter",
        "christians are the worst"
        ]

#Our final prediction
predict(model, texts)

Even for a small sample of texts, our model is working perfectly and predicting as expected.